Bayesian Optimization & NN Opt
---

In [1]:
from bayes_opt import BayesianOptimization
from nnopt.nnopt import Optimizer
import time
import numpy as np
import gym


In [2]:
ll = gym.make("LunarLanderContinuous-v2")
print(ll.action_space)
print(ll.observation_space)

def black_box_functionA(a, b, c, d, e, f, g, h, i, j, k, l, m, n, o, p, render=False):
    """Function with unknown internals we wish to maximize.

    This is just serving as an example, for all intents and
    purposes think of the internals of this function, i.e.: the process
    which generates its output values, as unknown.
    """
    transform = np.array([[a, b, c, d, e, f, g, h], [i, j, k, l, m, n, o, p]])
    
    sc = 0
    for _ in range(10):
        s = ll.reset()
        done = False
        while not done:
            if render:
                ll.render()
            s, r, done, _ = ll.step(np.clip(transform @ s, -1, 1))
            sc += r

    return sc / 4

def black_box_functionB(a, b):
    """Function with unknown internals we wish to maximize.

    This is just serving as an example, for all intents and
    purposes think of the internals of this function, i.e.: the process
    which generates its output values, as unknown.
    """
    return -a ** 2 - (b - 1) ** 2 + 1

#print(black_box_function(-221.59328, 28.267517))

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
Box(2,)
Box(8,)


/usr/local/lib/python3.6/site-packages/gym/envs/registration.py:14: DeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(False)


In [3]:
Nopt = Optimizer(black_box_functionA, N=16, surrogate_hidden_layer=400, kernel_common=400, kernel_seperate=400,
                 Rn=[(-10, 10), (-10, 10), (-10, 10), (-10, 10), (-10, 10), (-10, 10), (-10, 10), (-10, 10), (-10, 10), (-10, 10), (-10, 10), (-10, 10), (-10, 10), (-10, 10), (-10, 10), (-10, 10)]) #(-10, 10), (-10, 10), (-10, 10), (-10, 10)])

Bopt = BayesianOptimization(
    f=black_box_functionA,
    pbounds={'a': (-10, 10), 'b': (-10, 10), 'c': (-10, 10), 
             'd': (-10, 10), 'e': (-10, 10), 'f': (-10, 10), 'f': (-10, 10), 'g': (-10, 10), 'h': (-10, 10), 'i': (-10, 10), 'j': (-10, 10), 'k': (-10, 10), 'l': (-10, 10), 'm': (-10, 10), 'n': (-10, 10), 'o': (-10, 10), 'p': (-10, 10)},
    
    random_state=np.random.randint(10),
    verbose=2
)

Bayesian
---

In [4]:
timestamp = time.time()
Bopt.maximize(
    init_points=40,
    n_iter=2,
)

print("Bayesian took: {}".format(time.time() - timestamp))

|   iter    |  target   |     a     |     b     |     c     |     d     |     e     |     f     |     g     |     h     |     i     |     j     |     k     |     l     |     m     |     n     |     o     |     p     |
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|  1        | -2.304e+0 |  1.016    |  4.163    | -4.182    |  0.2166   |  7.859    |  7.926    | -7.488    | -5.855    | -8.971    | -1.184    | -9.402    | -0.8633   |  2.983    | -4.43     |  3.525    |  1.817    |
|  2        | -2.305e+0 | -9.52     |  1.177    | -4.815    | -1.698    | -4.329    |  3.863    | -1.191    | -6.863    |  0.893    |  5.606    | -3.873    | -5.561    | -2.241    |  8.728    |  9.52     |  3.448    |
|  3        | -1.362e+0 |  8.057    |  6.915    | -2.44     | -8.156    |  3.068    |  1.157    | -2.769    | -5.499    | -1.87 

|  23       | -1.643e+0 | -6.145    |  4.523    |  0.8796   |  0.6985   | -0.7093   | -9.866    | -2.151    |  9.979    |  6.406    |  1.457    | -6.071    |  8.409    | -1.517    |  2.387    | -5.938    | -3.439    |
|  24       | -1.19e+03 | -3.101    | -2.105    | -9.555    |  3.207    | -6.6      |  2.066    |  2.392    |  3.958    | -4.161    | -4.159    |  5.111    | -2.236    | -6.951    |  8.105    | -9.738    |  7.73     |
|  25       | -2.469e+0 | -4.155    | -2.363    |  9.568    | -1.266    |  4.318    | -1.752    |  8.271    |  4.39     | -0.3862   | -8.316    | -2.727    | -0.1879   | -4.323    |  0.6928   | -5.807    | -0.1134   |
|  26       | -2.721e+0 |  0.5601   |  3.467    |  4.907    |  4.599    | -1.518    | -0.881    |  7.47     | -2.744    |  4.761    |  9.881    |  8.496    |  0.9847   | -3.898    |  2.446    |  3.686    |  0.5392   |
|  27       | -2.323e+0 | -3.17     | -0.8915   | -8.364    | -4.174    | -7.102    |  3.33     |  9.185    | -6.855    | -9.434

In [5]:
timestamp = time.time()
Nopt.forget()
best, target = Nopt.run(random=20, optimization=5, fitting=2000, exploration=0.02,verbose=True)

print("Best", best, "target", target)
print(black_box_functionA(*best))
print("NNopt took:", time.time() - timestamp)

domain [-6.1983895   1.8514585  -7.3695803  -0.63046455 -6.796534   -7.8389764
 -2.217505   -2.9946613  -6.0795474   9.649033    2.760994   -8.110439
  0.38174438  9.321514    6.7139015   9.814606  ] target -2144.0719950289185
domain [ 4.9542217  -8.862176   -0.8499193  -7.6444435  -5.7598996   7.3427353
  3.799491    0.38357258  2.26686     4.4630957  -4.4713116   0.4241295
  9.852131   -4.298582   -8.377147   -3.876679  ] target -854.915608867391
domain [ 3.5165644  3.6195707 -4.0072155  6.028797  -0.6231785 -3.1275225
 -9.320998  -3.3970642  5.8453035  2.058978  -3.2671905  4.940979
 -2.7635098  4.6277237 -4.8084188  9.633457 ] target -1538.2865235201707
domain [-6.1992526   0.41101933  1.830864   -4.3823338  -1.303544    5.671179
 -4.8029256   1.8503113   0.61930656 -1.5861511  -3.9426208   0.67300034
 -5.699811    0.9338207   7.4133205  -8.247132  ] target -2766.171029555277
domain [ 6.844242   -3.606429   -3.4006      4.826004   -2.276938    6.0379677
  9.468174   -9.112827   -0.

In [ ]:

black_box_functionA(*best, render=True)

In [ ]:
# Stats from earlier opt 
W = [6.2106743, -10.0,        2.311772,  -10,         10,         7.4984336,
  10,        10,        -10,         10,       -10,         10,
  10,        10,        10,        -10,]       
black_box_functionA(*W, render=True)

In [ ]:
a =[ 0.64502907, -0.90281296, -5.945456,   -5.808859,    8.418125,    2.147026,
  5.653866,   -8.538346,   -6.319692,    1.7746944,  -1.2864513,   8.868685,
  6.0241585,   4.775818,    7.363823,   -7.1539164, ] 
black_box_functionA(*a, render=True)